In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)

In [3]:
stats = pd.read_csv('Web-Scrapping Scripts/Basketball-Reference per game stats 2017-19.csv')
print(stats.shape)
stats.head()

(1967, 30)


,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFT%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,Álex Abrines,SG,23,OKC,68,6,15.5,2.0,5.0,0.393,1.4,3.6,0.381,0.6,1.4,0.426,0.531,0.6,0.7,0.898,0.3,1.0,1.3,0.6,0.5,0.1,0.5,1.7,6.0,2017
1,Quincy Acy,PF,26,TOT,38,1,14.7,1.8,4.5,0.412,1.0,2.4,0.411,0.9,2.1,0.413,0.521,1.2,1.6,0.750,0.5,2.5,3.0,0.5,0.4,0.4,0.6,1.8,5.8,2017
2,Quincy Acy,PF,26,DAL,6,0,8.0,0.8,2.8,0.294,0.2,1.2,0.143,0.7,1.7,0.400,0.324,0.3,0.5,0.667,0.3,1.0,1.3,0.0,0.0,0.0,0.3,1.5,2.2,2017
3,Quincy Acy,PF,26,BRK,32,1,15.9,2.0,4.8,0.425,1.1,2.6,0.434,0.9,2.2,0.414,0.542,1.3,1.8,0.754,0.6,2.8,3.3,0.6,0.4,0.5,0.6,1.8,6.5,2017
4,Steven Adams,C,23,OKC,80,80,29.9,4.7,8.2,0.571,0.0,0.0,0.000,4.7,8.2,0.572,0.571,2.0,3.2,0.611,3.5,4.2,7.7,1.1,1.1,1.0,1.8,2.4,11.3,2017


# Data Cleaning 

Due to trades, some players have multiple entries. The 'Tm' features highlights which team the player has played for. TOT indicates the combined stats in one season while the other entries are stats for the respective teams they played in. We will consider the stats of the entire season. We will also be considering only players who have played at least 15 games.

In [7]:
#Replacing TOT with the last team played
for y in [2017, 2018, 2019]:
    dup = []
    for i, k in enumerate(stats[stats['Year'] == y]['Player']):
        dup += [k] if stats.loc[stats['Year'] == y, 'Tm'].reset_index(drop = True)[i] == 'TOT' else []

    for j in dup:
        index = list(stats.loc[(stats['Player'] == j) & (stats['Year'] == y), 'Tm'].index)
        stats.loc[index[0], 'Tm'] = stats.loc[index[-1], 'Tm']
        
stats.drop_duplicates(['Player', 'Year'], keep = 'first', inplace = True)

In [8]:
#Dropping players who played less than 15 games
stats = stats.loc[stats.G >= 15]
stats.reset_index(drop = True, inplace = True)

# Missing Data:

In [9]:
stats.isnull().sum()[stats.isnull().sum() > 0]

3P%    66
2P%     1
FT%     3
dtype: int64

In [11]:
stats[['3P', '3PA', '3P%']][stats['3P%'].isnull()].describe()

,3P,3PA,3P%
count,66.0,66.0,0.0
mean,0.0,0.0,NaN
std,0.0,0.0,NaN
min,0.0,0.0,NaN
25%,0.0,0.0,NaN
50%,0.0,0.0,NaN
75%,0.0,0.0,NaN
max,0.0,0.0,NaN


In [12]:
stats[['2P', '2PA', '2P%']][stats['2P%'].isnull()].describe()

,2P,2PA,2P%
count,1.0,1.0,0.0
mean,0.0,0.0,NaN
std,NaN,NaN,NaN
min,0.0,0.0,NaN
25%,0.0,0.0,NaN
50%,0.0,0.0,NaN
75%,0.0,0.0,NaN
max,0.0,0.0,NaN


In [13]:
stats[['FT', 'FTA', 'FT%']][stats['FT%'].isnull()].describe()

,FT,FTA,FT%
count,3.0,3.0,0.0
mean,0.0,0.0,NaN
std,0.0,0.0,NaN
min,0.0,0.0,NaN
25%,0.0,0.0,NaN
50%,0.0,0.0,NaN
75%,0.0,0.0,NaN
max,0.0,0.0,NaN


NAN because is all 0